<a href="https://colab.research.google.com/github/spirosChv/smartNetsWorkshop/blob/main/neuron/practical3_part4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating dendrites - Part 4

## How inputs inputs interact in time

In this notebook we will see how inputs integrate when they are seperated in time.

In [ ]:
!pip install neuron --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h
h.load_file("stdrun.hoc")

In [ ]:
# @title Make nicer plots -- Execute this cell
def mystyle():
  """
  Create custom plotting style.

  Returns
  -------
  my_style : dict
      Dictionary with matplotlib parameters.

  """
  # color pallette
  style = {
      # Use LaTeX to write all text
      "text.usetex": False,
      "font.family": "DejaVu Sans",
      "font.weight": "bold",
      # Use 16pt font in plots, to match 16pt font in document
      "axes.labelsize": 16,
      "axes.titlesize": 20,
      "font.size": 16,
      # Make the legend/label fonts a little smaller
      "legend.fontsize": 14,
      "xtick.labelsize": 14,
      "ytick.labelsize": 14,
      "axes.linewidth": 2.5,
      "lines.markersize": 10.0,
      "lines.linewidth": 2.5,
      "xtick.major.width": 2.2,
      "ytick.major.width": 2.2,
      "axes.labelweight": "bold",
      "axes.spines.right": False,
      "axes.spines.top": False
  }

  return style


plt.style.use("seaborn-colorblind")
plt.rcParams.update(mystyle())

### In this exercise we will see how inputs can be amplified by active sodium conductances

In [ ]:
# Simulation parameters
tstop = 200  # ms
h.dt = 0.1  # ms
vinit = -65  # initial membrane potential

### Create a soma and one dendrite, define their anatomical and biophysical properties and connect them.

In [ ]:
soma = h.Section(name='soma')
dend0 = h.Section(name='dend0') 

# Define properties of the soma
soma.diam = 18  # diametr (um)
soma.L = 18 # length (um)
soma.Ra = 100  # axial resistance (Ohm * cm)
soma.cm = 1  # specific membrane capacitance (uF/cm^2)   
soma.nseg = 1	 # number of segments

# Insert passive (leak) channels
soma.insert('pas')
for seg in soma:
  soma.e_pas = -65  # leak reversal potential (mV)
  soma.g_pas = 0.0003  # leak maximum conductance (S/cm^2)

# Define properties of dend0
dend0.diam = 5  # diametr (um)
dend0.L = 700 # length (um)
dend0.Ra = 100  # axial resistance (Ohm * cm)
dend0.cm = 1  # specific membrane capacitance (uF/cm^2)   
dend0.nseg = 11  # number of segments

# Insert passive (leak) channels
dend0.insert('pas')
for seg in dend0:
  seg.e_pas = -65  # leak reversal potential (mV)
  seg.g_pas = 0.0003  # leak maximum conductance (S/cm^2)

# Connect the 0 point of dendrite 0 to the zero point of the soma
dend0.connect(soma(0), 0)

### Task 1: We will add three ExpSyn in the different parts of the dendrite. What will happen to the temporal summation of the inputs if they are activated sequentially with 10ms intervals? Do you expect the temporal order of activation to affect the output?

In [ ]:
#========== Synaptic stimulation.
# Place three synapses, one in the end, one in the middle and one in the begining of the dendrite.
syn0 = h.ExpSyn(dend0(0.9))
syn0.e = 0  # reversal potential (mV)
syn0.tau = 10  # decay time constant (ms)

syn1 = h.ExpSyn(dend0(0.5))
syn1.e = 0  # reversal potential (mV)
syn1.tau = 10  # decay time constant (ms)

syn2 = h.ExpSyn(dend0(0.1))
syn2.e = 0  # reversal potential (mV)
syn2.tau = 10  # decay time constant (ms)

#========== ...create an artificial spike (an "event" to be delivered to the synapse)...
tsignal = 20
ns = h.NetStim(0.5)
ns.start = tsignal
ns.number = 1

#... and connect the event to the synapses.
nc0 = h.NetCon(ns, syn0)
nc0.delay = 20
nc0.weight[0] = 0.003

nc1 = h.NetCon(ns, syn1)
nc1.delay = 10
nc1.weight[0] = 0.003

nc2 = h.NetCon(ns, syn2)
nc2.delay = 0
nc2.weight[0] = 0.003

In [ ]:
# Calculate and print in terminal the depolarization at the soma and at the middle part of the distal dendrite (dendrite[1])
vsoma_vec = h.Vector().record(soma(0.5)._ref_v) # Membrane potential vector
vdend0_vec = h.Vector().record(dend0(0.5)._ref_v) # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

# reinitialize the simulator and run again
h.finitialize(vinit)
h.continuerun(tstop)

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.plot(t_vec, vdend0_vec, color='red', label='dendrite')
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.legend()
plt.show()

print(f'\nSomatic depolarization is {round(vsoma_vec.max()-vsoma_vec[int((tsignal)/h.dt)-1], 2)} mV')